## Import Libraries

In [1]:
import os
import warnings

import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

import xgboost as xgb

from google.colab import drive

In [2]:
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
%matplotlib inline

In [3]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
RANDOM_STATE = 42

## Load Data

In [5]:
DATA_PATH = "gdrive/MyDrive/EASY_MONEY_NUCLIO/data/"

In [6]:
df = pd.read_csv(DATA_PATH + "total_df.csv", sep=";", index_col=0)

In [7]:
df["pk_partition"] = pd.to_datetime(df["pk_partition"], format="%Y-%m-%d")

In [8]:
PRODUCTS = [
    "short_term_deposit",	
    "loans",
    "mortgage",	
    "funds",
    "securities",
    "long_term_deposit",	
    "em_account_pp",	
    "credit_card",	
    "payroll",	
    "pension_plan",	
    "payroll_account",	
    "emc_account",	
    "debit_card",	
    "em_account_p",	
    "em_acount"	
]

PRICE = {
    "Finaciación": 60,
    "Ahorro/Inversion": 40,
    "Cuentas": 10
}

PRODUCTS_PRICED = {
    "short_term_deposit": "Ahorro/Inversion",	
    "loans": "Finaciación",
    "mortgage": "Finaciación",	
    "funds": "Ahorro/Inversion",
    "securities": "Ahorro/Inversion",
    "long_term_deposit": "Ahorro/Inversion",	
    "em_account_pp": "Cuentas",	
    "credit_card": "Finaciación",
    "pension_plan": "Ahorro/Inversion",	
    "payroll_account": "Cuentas",	
    "emc_account": "Cuentas",	
    "debit_card": "Finaciación",	
    "em_account_p": "Cuentas",	
    "em_acount": "Cuentas"	
}

# EDA

In [9]:
df.isna().sum()

pk_cid                      0
pk_partition                0
short_term_deposit          0
loans                       0
mortgage                    0
funds                       0
securities                  0
long_term_deposit           0
em_account_pp               0
credit_card                 0
payroll                    61
pension_plan               61
payroll_account             0
emc_account                 0
debit_card                  0
em_account_p                0
em_acount                   0
entry_date                  0
entry_channel          133033
active_customer             0
segment                133944
country_id                  0
region_code              2264
gender                     25
age                         0
deceased                    0
salary                1512103
dtype: int64

In [10]:
df[["pk_cid", "pk_partition"]].duplicated().astype(int).sum()

0

# NaN's

In [11]:
df["salary"] = df["salary"].fillna(-9999)

In [12]:
df["region_code"] = df["region_code"].fillna(-99)

In [13]:
df["entry_channel"] = df["entry_channel"].fillna("Unknown")

In [14]:
df["pension_plan"].value_counts(dropna=False)

0.0    5745061
1.0     217802
NaN         61
Name: pension_plan, dtype: int64

In [15]:
df["pension_plan"] = df["pension_plan"].fillna(0.0)

In [16]:
df.isna().sum()

pk_cid                     0
pk_partition               0
short_term_deposit         0
loans                      0
mortgage                   0
funds                      0
securities                 0
long_term_deposit          0
em_account_pp              0
credit_card                0
payroll                   61
pension_plan               0
payroll_account            0
emc_account                0
debit_card                 0
em_account_p               0
em_acount                  0
entry_date                 0
entry_channel              0
active_customer            0
segment               133944
country_id                 0
region_code                0
gender                    25
age                        0
deceased                   0
salary                     0
dtype: int64

# Preprocessing

In [17]:
df["deceased"].value_counts()

N    5961849
S       1075
Name: deceased, dtype: int64

No podemos recomendar productos a clientes que estan muertos, por tanto, filtramos estos clientes.

In [18]:
df = df[df["deceased"] != "S"]

In [19]:
df = df[(df["age"] >= 18) & (df["age"] <= 90)]

In [20]:
df.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,entry_date,entry_channel,active_customer,segment,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,2018-01-12,KHL,1.0,02 - PARTICULARES,ES,29.0,H,35,N,87218.10
1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,2015-08-10,KHE,0.0,03 - UNIVERSITARIO,ES,13.0,V,23,N,35548.74
2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,2015-08-10,KHE,0.0,03 - UNIVERSITARIO,ES,13.0,V,23,N,122179.11
3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,2015-08-10,KHD,0.0,03 - UNIVERSITARIO,ES,50.0,H,22,N,119775.54
4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,2015-08-10,KHE,1.0,03 - UNIVERSITARIO,ES,50.0,V,23,N,-9999.00


In [21]:
cols_to_drop = ["entry_date", "segment", "deceased", "payroll", "gender"]
df = df.drop(cols_to_drop, axis=1)

In [22]:
print(df.shape)
print(df.columns)

(5921197, 22)
Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'pension_plan', 'payroll_account', 'emc_account',
       'debit_card', 'em_account_p', 'em_acount', 'entry_channel',
       'active_customer', 'country_id', 'region_code', 'age', 'salary'],
      dtype='object')


In [23]:
df["country_id"] = LabelEncoder().fit_transform(df["country_id"])

In [24]:
df["year"] = df["pk_partition"].dt.year
df["month"] = df["pk_partition"].dt.month
del df["pk_partition"]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5921197 entries, 0 to 5962923
Data columns (total 23 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   short_term_deposit  int64  
 2   loans               int64  
 3   mortgage            int64  
 4   funds               int64  
 5   securities          int64  
 6   long_term_deposit   int64  
 7   em_account_pp       int64  
 8   credit_card         int64  
 9   pension_plan        float64
 10  payroll_account     int64  
 11  emc_account         int64  
 12  debit_card          int64  
 13  em_account_p        int64  
 14  em_acount           int64  
 15  entry_channel       object 
 16  active_customer     float64
 17  country_id          int64  
 18  region_code         float64
 19  age                 int64  
 20  salary              float64
 21  year                int64  
 22  month               int64  
dtypes: float64(4), int64(18), object(1)
memory usage: 1.1+ GB

In [26]:
def set_others(df, col, n):
    """
    Converts to Categorical column and sets Others for the categories outside the top n
    :param df: DataFrame object
    :param col: column name
    :param n: top X
    :return 
    """
    top_categories = df[col].value_counts().head(n)
    top_categories_list = top_categories.index.tolist()
    top_categories_list.append("Others")

    df[col] = pd.Categorical(df[col], categories=top_categories_list)
    return df[col].fillna("Others")

In [27]:
df["entry_channel"].value_counts().head(8)

KHE        3113575
KFC         873049
KHQ         590224
KAT         405973
KHK         229826
KHM         176333
Unknown     132171
KHN         108157
Name: entry_channel, dtype: int64

In [28]:
df["entry_channel"] = set_others(df, "entry_channel", 8)

In [29]:
def one_hot_encoder(data, col):
    _dummy_dataset = pd.get_dummies(data[col], prefix=col)
    data = pd.concat([data, _dummy_dataset], axis=1)
    del data[col]
    del _dummy_dataset

    return data

In [30]:
df = one_hot_encoder(df, "entry_channel")

In [31]:
# Region code frequency encoding
region_counter = df["region_code"].value_counts()
df["region_code"] = df["region_code"].replace(region_counter)
df["region_code"]

0           187309.0
1            69905.0
2            69905.0
3           132590.0
4           132590.0
             ...    
5962919     132590.0
5962920      34660.0
5962921     132590.0
5962922     132590.0
5962923    1162770.0
Name: region_code, Length: 5921197, dtype: float64

# Modelling

In [32]:
df.columns

Index(['pk_cid', 'short_term_deposit', 'loans', 'mortgage', 'funds',
       'securities', 'long_term_deposit', 'em_account_pp', 'credit_card',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_account_p', 'em_acount', 'active_customer', 'country_id',
       'region_code', 'age', 'salary', 'year', 'month', 'entry_channel_KHE',
       'entry_channel_KFC', 'entry_channel_KHQ', 'entry_channel_KAT',
       'entry_channel_KHK', 'entry_channel_KHM', 'entry_channel_Unknown',
       'entry_channel_KHN', 'entry_channel_Others'],
      dtype='object')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5921197 entries, 0 to 5962923
Data columns (total 31 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   pk_cid                 int64  
 1   short_term_deposit     int64  
 2   loans                  int64  
 3   mortgage               int64  
 4   funds                  int64  
 5   securities             int64  
 6   long_term_deposit      int64  
 7   em_account_pp          int64  
 8   credit_card            int64  
 9   pension_plan           float64
 10  payroll_account        int64  
 11  emc_account            int64  
 12  debit_card             int64  
 13  em_account_p           int64  
 14  em_acount              int64  
 15  active_customer        float64
 16  country_id             int64  
 17  region_code            float64
 18  age                    int64  
 19  salary                 float64
 20  year                   int64  
 21  month                  int64  
 22  entry_channel_KHE 

In [34]:
TARGETS = [
            "short_term_deposit", 
            "loans", 
            "mortgage", 
            "funds",
            "securities", 
            "long_term_deposit", 
            "em_account_pp", 
            "credit_card",
            "pension_plan", 
            "payroll_account", 
            "emc_account", 
            "debit_card",
            "em_account_p", 
            "em_acount"
]

In [35]:
X = df.drop(TARGETS + ["pk_cid"], axis=1)
y = df[TARGETS]

In [36]:
df.shape

(5921197, 31)

In [37]:
# Validation and Train-Test
X_train_test, X_validation, y_train_test, y_validation =  train_test_split(X, y, test_size=0.1, random_state=RANDOM_STATE)

# Train and Test
X_train, X_test, y_train, y_test =  train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=RANDOM_STATE)

In [38]:
model = MultiOutputClassifier(xgb.XGBClassifier(
    max_depth=4,
    min_samples_leaf = 200,
    random_state=RANDOM_STATE
))

In [ ]:
model.fit(X_train, y_train)

[10:20:20] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [ ]:
y_train_pred = dt.predict(X_train)
y_train_score = dt.predict_proba(X_train)[:,1]

#Predict the response for test dataset
y_pred = dt.predict(X_test)
y_test_score = dt.predict_proba(X_test)[:,1]